In [1]:
import os

In [2]:
%pwd

'c:\\Users\\nguye\\Documents\\AI\\Youtube_tutorials\\end-to-end-ml-project\\research'

In [3]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\nguye\\Documents\\AI\\Youtube_tutorials\\end-to-end-ml-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [8]:
from ml_in_action.constants import *
from ml_in_action.utils.common import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [14]:
import os
from ml_in_action import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [22]:
class DataTransformation:
    def __init__(self, config):
        self.config = config

    def train_test_spliting(self):
        # Load dataset
        data = pd.read_csv(self.config.data_path)

        # Drop the last column (assuming it's named 'Id')
        if 'Id' in data.columns:
            data = data.drop(columns=['Id'])

        # Splitting features and target variable
        X = data.iloc[:, :-1]  # All columns except last (features)
        y = data.iloc[:, -1]   # Last column (target)

        # Train-test split (80% train, 20% test)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Apply feature scaling
        scaler = StandardScaler()  # Use MinMaxScaler() if needed
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Convert back to DataFrame
        train = pd.DataFrame(X_train_scaled, columns=X.columns)
        train['target'] = y_train.values  # Add target back

        test = pd.DataFrame(X_test_scaled, columns=X.columns)
        test['target'] = y_test.values  # Add target back

        # Save processed data
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"), index=False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"), index=False)

        logger.info("Split data into training and test sets")
        logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")

        print(f"Train shape: {train.shape}, Test shape: {test.shape}")

In [23]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-03-15 16:01:56,725: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-15 16:01:56,726: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-15 16:01:56,728: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-15 16:01:56,729: INFO: common: created directory at: artifacts]
[2025-03-15 16:01:56,730: INFO: common: created directory at: artifacts/data_transformation]
[2025-03-15 16:01:56,748: INFO: 2747417385: Split data into training and test sets]
[2025-03-15 16:01:56,749: INFO: 2747417385: Train shape: (914, 12), Test shape: (229, 12)]
Train shape: (914, 12), Test shape: (229, 12)
